## Prepare environment

Install libraries

In [ ]:
%%capture
!pip install git+https://github.com/huggingface/transformers.git  -U
!pip install git+https://github.com/huggingface/accelerate.git  -U
!pip install bitsandbytes
!pip install git+https://github.com/huggingface/peft.git  -U

Import libraries

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch
from peft import PeftModel, PeftConfig, get_peft_model, PromptTuningInit, PromptTuningConfig, PrefixTuningConfig, LoraConfig, TaskType, PeftType, PeftConfig, PeftModel
# prepare_model_for_int8_training
import time
# import evaluate
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yaml

## Read data

Read data from Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/data/table_insights_labeled_data.xlsx', sheet_name="Data")

In [ ]:
dataset

,DeviceMaskedID,data (train/ test),date,cpu_usage,ram_usage,diskio_usage
0,Device_1,Train,2022-08-02,7.845169,52.662081,NaN
1,Device_1,Train,2022-08-03,5.029416,53.519746,NaN
2,Device_1,Train,2022-08-04,4.855019,53.656122,NaN
3,Device_1,Train,2022-08-05,8.014844,55.245124,NaN
4,Device_1,Train,2022-08-07,16.909919,54.216115,NaN
...,...,...,...,...,...,...
5276,Device_215,Train,2022-05-25,NaN,NaN,90.005963
5277,Device_215,Train,2022-05-26,NaN,NaN,75.829685
5278,Device_215,Train,2022-05-27,NaN,NaN,88.540792
5279,Device_215,Train,2022-05-28,NaN,NaN,77.175451


In [ ]:
lables = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/data/table_insights_labeled_data.xlsx', sheet_name="Labels")

In [ ]:
lables

,DeviceMaskedID,data (train/ test),insight_0,insight_1,insight_2
0,Device_1,Train,The data contains information about a computer...,1. The highest CPU usage was recorded on 222-8...,1. The diskio_usage column has no data (NaN) f...
1,Device_2,Train,The data contains CPU usage information for a ...,1. CPU usage has been consistently increasing ...,"Based on the provided data, the top 3 aberrati..."
2,Device_3,Train,The data contains information about a user's c...,1. The highest CPU usage was recorded on 223-3...,1. High CPU usage on 223-3-13: The CPU usage o...
3,Device_4,Test,The data contains information about a user's c...,1. High CPU usage days: The user experienced t...,1. Missing Data: The 'diskio_usage' column has...
4,Device_5,Train,The data contains information about a user's c...,1. High CPU Usage: The highest CPU usage recor...,1. Missing Data: The 'diskio_usage' column has...
...,...,...,...,...,...
210,Device_211,Train,The data contains information about a user's c...,1. High RAM Usage: The RAM usage is consistent...,"1. Highest CPU usage: On 223-3-1, the CPU usag..."
211,Device_212,Train,The data contains information about a user's c...,1. Significant increase in resource usage towa...,1. There is a significant increase in CPU usag...
212,Device_213,Test,The data contains information about a user's c...,1. Highest CPU usage: The highest CPU usage wa...,1. Missing Data: The 'diskio_usage' column has...
213,Device_214,Train,The data contains daily disk IO usage for a co...,1. The highest disk IO usage was recorded on 1...,"After analyzing the dataset, the top 3 aberrat..."


In [ ]:
data_dictionary = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/data/table_insights_labeled_data.xlsx', sheet_name="Data dictionary")

In [ ]:
dataset.columns

Index(['DeviceMaskedID', 'data (train/ test)', 'date', 'cpu_usage',
       'ram_usage', 'diskio_usage'],
      dtype='object')

## Split train and test data

In [ ]:
dataset["data (train/ test)"].unique()

array(['Train', 'Test'], dtype=object)

In [ ]:
dataset["data (train/ test)"].value_counts()

data (train/ test)
Train    4288
Test      993
Name: count, dtype: int64

In [ ]:
train = dataset.loc[dataset["data (train/ test)"] == "Train"]

In [ ]:
test = dataset.loc[dataset["data (train/ test)"] == "Train"]

## EDA

In [ ]:
train.dtypes

DeviceMaskedID                object
data (train/ test)            object
date                  datetime64[ns]
cpu_usage                    float64
ram_usage                    float64
diskio_usage                 float64
dtype: object

## Load Model

Model Config

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.float16,
    bnb_4bit_use_double_quant= True,
llm_int8_enable_fp32_cpu_offload= True)

In [ ]:
# model_name = 'databricks/dolly-v2-3b'
model_name = 'mistralai/Mistral-7B-Instruct-v0.2'

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
ex_device_id = train["DeviceMaskedID"][0]
ex_insight = lables.loc[lables["DeviceMaskedID"]==ex_device_id]["insight_2"][0]

In [ ]:
ex_insight

'1. The diskio_usage column has no data (NaN) for all the entries, which indicates missing or incomplete information about disk io usage.\n\n2. The CPU usage shows a significant spike on 222-8-7 with a value of 16.99919, which is much higher than the average CPU usage observed in the dataset.\n\n3. The RAM usage reaches its highest value of 72.26848 on 222-8-26, which is notably higher than the average RAM usage in the dataset.'

## Prompt Engineering

Check different input types

In [ ]:
df = train.loc[train["DeviceMaskedID"]==ex_device_id]
df = df.loc[:, ~df.columns.isin(["DeviceMaskedID", "data (train/ test)"])]
df["date"] = df["date"].dt.strftime('%Y-%m-%d')
df = df.round(2)
# df = df.fillna("")
## Json type
# df_str = df.to_json(orient="index")
train_prompt = df.to_json(orient="records", lines=True)
# df_str = df.to_csv(sep=",")
# df_str = df.to_json(orient="split")

# YAML type
# df_str = yaml.dump(
#   df.reset_index().to_dict(orient='records'),
#   # sort_keys=False, width=72, indent=4,
#   # default_flow_style=None)
# )

# df_str = df_str.replace('"', '')
# df_str = df_str.replace('{', '')
# df_str = df_str.replace('}', '')
# paragraph type
# for index, row in df.iterrows():
# df_str = ""
# for ind in df.index:
#   df_str =  f"{df_str}On {df['date'][ind]} cpu usage is {df['cpu_usage'][ind]} and ram usage is {df['ram_usage'][ind]} {'and diskio_usage is ' + str(df['diskio_usage'][ind]) if df['diskio_usage'][ind] else ''}. "
print(train_prompt)


{"date":"2022-08-02","cpu_usage":7.85,"ram_usage":52.66,"diskio_usage":null}
{"date":"2022-08-03","cpu_usage":5.03,"ram_usage":53.52,"diskio_usage":null}
{"date":"2022-08-04","cpu_usage":4.86,"ram_usage":53.66,"diskio_usage":null}
{"date":"2022-08-05","cpu_usage":8.01,"ram_usage":55.25,"diskio_usage":null}
{"date":"2022-08-07","cpu_usage":16.91,"ram_usage":54.22,"diskio_usage":null}
{"date":"2022-08-08","cpu_usage":6.97,"ram_usage":59.46,"diskio_usage":null}
{"date":"2022-08-09","cpu_usage":6.95,"ram_usage":62.01,"diskio_usage":null}
{"date":"2022-08-10","cpu_usage":6.91,"ram_usage":62.33,"diskio_usage":null}
{"date":"2022-08-11","cpu_usage":6.81,"ram_usage":61.52,"diskio_usage":null}
{"date":"2022-08-12","cpu_usage":7.5,"ram_usage":63.4,"diskio_usage":null}
{"date":"2022-08-15","cpu_usage":5.93,"ram_usage":64.43,"diskio_usage":null}
{"date":"2022-08-16","cpu_usage":9.36,"ram_usage":60.28,"diskio_usage":null}
{"date":"2022-08-17","cpu_usage":9.72,"ram_usage":63.67,"diskio_usage":null}


In [ ]:
train

,DeviceMaskedID,data (train/ test),date,cpu_usage,ram_usage,diskio_usage
0,Device_1,Train,2022-08-02,7.845169,52.662081,NaN
1,Device_1,Train,2022-08-03,5.029416,53.519746,NaN
2,Device_1,Train,2022-08-04,4.855019,53.656122,NaN
3,Device_1,Train,2022-08-05,8.014844,55.245124,NaN
4,Device_1,Train,2022-08-07,16.909919,54.216115,NaN
...,...,...,...,...,...,...
5276,Device_215,Train,2022-05-25,NaN,NaN,90.005963
5277,Device_215,Train,2022-05-26,NaN,NaN,75.829685
5278,Device_215,Train,2022-05-27,NaN,NaN,88.540792
5279,Device_215,Train,2022-05-28,NaN,NaN,77.175451


Finalized jason input types

Prompt Engineering with Zero shot

In [ ]:
# print(train["DeviceMaskedID"].unique())a
count = 0
for device_id in train["DeviceMaskedID"].unique():
  if device_id == ex_device_id:
    continue
  count = count + 1
  print(device_id)
  df = train.loc[train["DeviceMaskedID"]==device_id]
  df = df.loc[:, ~df.columns.isin(["DeviceMaskedID", "data (train/ test)"])]
  df["date"] = df["date"].dt.strftime('%Y-%m-%d')
  df = df.round(2)
  # df = df.fillna("")

  df_str = df.to_json(orient="records", lines=True)

  # Zero shot
  prompt_zero_shot = f"""
  Analyze the following table data and provide a creative summary in not more than 3 points, without including averages or mathematical computations, while maintaining the accuracy of the provided numbers:

  {df_str}

  Summary:
  """

  # Zero Shot
  inputs_zero_shot = tokenizer(prompt_zero_shot, return_tensors='pt')
  output_zero_shot = tokenizer.decode(
      model.generate(
          **inputs_zero_shot.to("cuda"),
          max_new_tokens=800,
          do_sample=False,
      )[0],
      skip_special_tokens=True
  )

  dash_line = '-'.join('' for x in range(100))
  print(dash_line)
  # print(f'INPUT PROMPT:\n{prompt}')
  # print(dash_line)
  print(f'MODEL GENERATION - ZERO SHOT:\n{output_zero_shot}')
  if count == 5:
    break

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Device_2


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:

  Analyze the following table data and provide a creative summary in not more than 3 points, without including averages or mathematical computations, while maintaining the accuracy of the provided numbers:

  {"date":"2022-01-09","cpu_usage":27.04,"ram_usage":null,"diskio_usage":null}
{"date":"2022-01-10","cpu_usage":46.86,"ram_usage":null,"diskio_usage":null}
{"date":"2022-01-11","cpu_usage":43.94,"ram_usage":null,"diskio_usage":null}
{"date":"2022-01-12","cpu_usage":48.29,"ram_usage":null,"diskio_usage":null}
{"date":"2022-01-13","cpu_usage":64.99,"ram_usage":null,"diskio_usage":null}
{"date":"2022-01-14","cpu_usage":71.53,"ram_usage":null,"diskio_usage":null}
{"date":"2022-01-15","cpu_usage":64.7,"ram_usage":null,"diskio_usage":null}
{"date":"2022-01-16","cpu_usage":83.23,"ram_usage":null,"diskio_usage":null}
{"date":"2022-01-17","cpu_usage":78.5,"ram_us

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:

  Analyze the following table data and provide a creative summary in not more than 3 points, without including averages or mathematical computations, while maintaining the accuracy of the provided numbers:

  {"date":"2023-03-13","cpu_usage":62.14,"ram_usage":53.12,"diskio_usage":12.34}
{"date":"2023-03-14","cpu_usage":20.91,"ram_usage":65.4,"diskio_usage":0.93}
{"date":"2023-03-15","cpu_usage":23.32,"ram_usage":68.72,"diskio_usage":1.54}
{"date":"2023-03-16","cpu_usage":26.29,"ram_usage":69.52,"diskio_usage":2.03}
{"date":"2023-03-17","cpu_usage":19.4,"ram_usage":70.42,"diskio_usage":0.58}
{"date":"2023-03-20","cpu_usage":41.81,"ram_usage":68.42,"diskio_usage":5.33}
{"date":"2023-03-21","cpu_usage":19.44,"ram_usage":66.25,"diskio_usage":0.85}
{"date":"2023-03-22","cpu_usage":29.11,"ram_usage":62.6,"diskio_usage":1.76}
{"date":"2023-03-23","cpu_usage":18.96

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:

  Analyze the following table data and provide a creative summary in not more than 3 points, without including averages or mathematical computations, while maintaining the accuracy of the provided numbers:

  {"date":"2022-08-01","cpu_usage":22.0,"ram_usage":36.81,"diskio_usage":null}
{"date":"2022-08-02","cpu_usage":11.47,"ram_usage":40.06,"diskio_usage":null}
{"date":"2022-08-04","cpu_usage":37.75,"ram_usage":50.06,"diskio_usage":null}
{"date":"2022-08-05","cpu_usage":16.51,"ram_usage":31.5,"diskio_usage":null}
{"date":"2022-08-08","cpu_usage":35.99,"ram_usage":62.27,"diskio_usage":null}
{"date":"2022-08-09","cpu_usage":40.92,"ram_usage":44.9,"diskio_usage":null}
{"date":"2022-08-10","cpu_usage":58.47,"ram_usage":35.26,"diskio_usage":null}
{"date":"2022-08-11","cpu_usage":54.47,"ram_usage":31.4,"diskio_usage":null}
{"date":"2022-08-12","cpu_usage":62.95,"

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:

  Analyze the following table data and provide a creative summary in not more than 3 points, without including averages or mathematical computations, while maintaining the accuracy of the provided numbers:

  {"date":"2023-03-01","cpu_usage":65.41,"ram_usage":45.86,"diskio_usage":1.48}
{"date":"2023-03-02","cpu_usage":66.49,"ram_usage":49.08,"diskio_usage":1.59}
{"date":"2023-03-03","cpu_usage":65.22,"ram_usage":47.63,"diskio_usage":1.48}
{"date":"2023-03-04","cpu_usage":65.44,"ram_usage":48.16,"diskio_usage":1.42}
{"date":"2023-03-05","cpu_usage":63.62,"ram_usage":48.88,"diskio_usage":1.34}
{"date":"2023-03-06","cpu_usage":64.5,"ram_usage":49.22,"diskio_usage":1.32}
{"date":"2023-03-07","cpu_usage":65.05,"ram_usage":47.95,"diskio_usage":1.45}
{"date":"2023-03-08","cpu_usage":67.68,"ram_usage":62.59,"diskio_usage":1.61}
{"date":"2023-03-09","cpu_usage":91.8

Prompt Engineering with One shot with only summary without providing example table

In [ ]:
count = 0
for device_id in train["DeviceMaskedID"].unique():
  if device_id == ex_device_id:
    continue
  count = count + 1
  print(device_id)
  df = train.loc[train["DeviceMaskedID"]==device_id]
  df = df.loc[:, ~df.columns.isin(["DeviceMaskedID", "data (train/ test)"])]
  df["date"] = df["date"].dt.strftime('%Y-%m-%d')
  df = df.round(2)
  # df = df.fillna("")

  df_str = df.to_json(orient="records", lines=True)

  # One shot without example table
  prompt_one_shot_wo_table = f"""
  Summary:
  {ex_insight}


  Take the above summary as example only and analyze the following table data and provide a creative summary in not more than 3 points, without including averages or mathematical computations, while maintaining the accuracy of the provided numbers:

  {df_str}

  Summary:
  """

  inputs_one_shot_wo_table = tokenizer(prompt_one_shot_wo_table, return_tensors='pt')
  output_one_shot_wo_table = tokenizer.decode(
      model.generate(
          **inputs_one_shot_wo_table.to("cuda"),
          max_new_tokens=800,
          do_sample=False,
          # temperature=0.5,
          # top_k=100,
          # top_p=0.8,
      )[0],
      skip_special_tokens=True
  )

  dash_line = '-'.join('' for x in range(100))
  print(dash_line)
  # print(f'INPUT PROMPT:\n{prompt}')
  # print(dash_line)
  print(f'MODEL GENERATION - ONE SHOT WITHOUT TABLE:\n{output_one_shot_wo_table}')

  if count == 5:
    break

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Device_2


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT WITHOUT TABLE:

  Summary:
  1. The diskio_usage column has no data (NaN) for all the entries, which indicates missing or incomplete information about disk io usage.

2. The CPU usage shows a significant spike on 222-8-7 with a value of 16.99919, which is much higher than the average CPU usage observed in the dataset.

3. The RAM usage reaches its highest value of 72.26848 on 222-8-26, which is notably higher than the average RAM usage in the dataset.


  Take the above summary as example only and analyze the following table data and provide a creative summary in not more than 3 points, without including averages or mathematical computations, while maintaining the accuracy of the provided numbers:

  {"date":"2022-01-09","cpu_usage":27.04,"ram_usage":null,"diskio_usage":null}
{"date":"2022-01-10","cpu_usage":46.86,"ram_usage":null,"diskio_usage":null}
{"date":

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT WITHOUT TABLE:

  Summary:
  1. The diskio_usage column has no data (NaN) for all the entries, which indicates missing or incomplete information about disk io usage.

2. The CPU usage shows a significant spike on 222-8-7 with a value of 16.99919, which is much higher than the average CPU usage observed in the dataset.

3. The RAM usage reaches its highest value of 72.26848 on 222-8-26, which is notably higher than the average RAM usage in the dataset.


  Take the above summary as example only and analyze the following table data and provide a creative summary in not more than 3 points, without including averages or mathematical computations, while maintaining the accuracy of the provided numbers:

  {"date":"2023-03-13","cpu_usage":62.14,"ram_usage":53.12,"diskio_usage":12.34}
{"date":"2023-03-14","cpu_usage":20.91,"ram_usage":65.4,"diskio_usage":0.93}
{"date

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT WITHOUT TABLE:

  Summary:
  1. The diskio_usage column has no data (NaN) for all the entries, which indicates missing or incomplete information about disk io usage.

2. The CPU usage shows a significant spike on 222-8-7 with a value of 16.99919, which is much higher than the average CPU usage observed in the dataset.

3. The RAM usage reaches its highest value of 72.26848 on 222-8-26, which is notably higher than the average RAM usage in the dataset.


  Take the above summary as example only and analyze the following table data and provide a creative summary in not more than 3 points, without including averages or mathematical computations, while maintaining the accuracy of the provided numbers:

  {"date":"2022-08-01","cpu_usage":22.0,"ram_usage":36.81,"diskio_usage":null}
{"date":"2022-08-02","cpu_usage":11.47,"ram_usage":40.06,"diskio_usage":null}
{"date"

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT WITHOUT TABLE:

  Summary:
  1. The diskio_usage column has no data (NaN) for all the entries, which indicates missing or incomplete information about disk io usage.

2. The CPU usage shows a significant spike on 222-8-7 with a value of 16.99919, which is much higher than the average CPU usage observed in the dataset.

3. The RAM usage reaches its highest value of 72.26848 on 222-8-26, which is notably higher than the average RAM usage in the dataset.


  Take the above summary as example only and analyze the following table data and provide a creative summary in not more than 3 points, without including averages or mathematical computations, while maintaining the accuracy of the provided numbers:

  {"date":"2023-03-01","cpu_usage":65.41,"ram_usage":45.86,"diskio_usage":1.48}
{"date":"2023-03-02","cpu_usage":66.49,"ram_usage":49.08,"diskio_usage":1.59}
{"date

Prompt Engineering with One shot

In [ ]:
count = 0
for device_id in train["DeviceMaskedID"].unique():
  if device_id == ex_device_id:
    continue
  count = count + 1
  print(device_id)
  df = train.loc[train["DeviceMaskedID"]==device_id]
  df = df.loc[:, ~df.columns.isin(["DeviceMaskedID", "data (train/ test)"])]
  df["date"] = df["date"].dt.strftime('%Y-%m-%d')
  df = df.round(2)
  # df = df.fillna("")

  df_str = df.to_json(orient="records", lines=True)

  # One shot
  prompt_one_shot = f"""
  Summarize the following table data.

  {train_prompt}

  Summary:
  {ex_insight}

  Take the above summary as example only and analyze the following table data and provide a creative summary in not more than 3 points, without including averages or mathematical computations, while maintaining the accuracy of the provided numbers:

  {df_str}

  Summary:
  """

  inputs_one_shot = tokenizer(prompt_one_shot, return_tensors='pt')
  output_one_shot = tokenizer.decode(
      model.generate(
          **inputs_one_shot.to("cuda"),
          max_new_tokens=800,
          do_sample=False,
          # temperature=0.5,
          # top_k=100,
          # top_p=0.8,
          # typical_p=0.8,
          # num_beams=12,
          # num_beam_groups=3,
          # diversity_penalty=1.0
      )[0],
      skip_special_tokens=True
  )

  dash_line = '-'.join('' for x in range(100))
  print(dash_line)
  # print(f'INPUT PROMPT:\n{prompt}')
  # print(dash_line)
  print(f'MODEL GENERATION - ONE SHOT:\n{output_one_shot}')
  if count == 5:
    break

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Device_2


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:

  Summarize the following table data.

  {"date":"2022-08-02","cpu_usage":7.85,"ram_usage":52.66,"diskio_usage":null}
{"date":"2022-08-03","cpu_usage":5.03,"ram_usage":53.52,"diskio_usage":null}
{"date":"2022-08-04","cpu_usage":4.86,"ram_usage":53.66,"diskio_usage":null}
{"date":"2022-08-05","cpu_usage":8.01,"ram_usage":55.25,"diskio_usage":null}
{"date":"2022-08-07","cpu_usage":16.91,"ram_usage":54.22,"diskio_usage":null}
{"date":"2022-08-08","cpu_usage":6.97,"ram_usage":59.46,"diskio_usage":null}
{"date":"2022-08-09","cpu_usage":6.95,"ram_usage":62.01,"diskio_usage":null}
{"date":"2022-08-10","cpu_usage":6.91,"ram_usage":62.33,"diskio_usage":null}
{"date":"2022-08-11","cpu_usage":6.81,"ram_usage":61.52,"diskio_usage":null}
{"date":"2022-08-12","cpu_usage":7.5,"ram_usage":63.4,"diskio_usage":null}
{"date":"2022-08-15","cpu_usage":5.93,"ram_usage":64.43,"dis

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:

  Summarize the following table data.

  {"date":"2022-08-02","cpu_usage":7.85,"ram_usage":52.66,"diskio_usage":null}
{"date":"2022-08-03","cpu_usage":5.03,"ram_usage":53.52,"diskio_usage":null}
{"date":"2022-08-04","cpu_usage":4.86,"ram_usage":53.66,"diskio_usage":null}
{"date":"2022-08-05","cpu_usage":8.01,"ram_usage":55.25,"diskio_usage":null}
{"date":"2022-08-07","cpu_usage":16.91,"ram_usage":54.22,"diskio_usage":null}
{"date":"2022-08-08","cpu_usage":6.97,"ram_usage":59.46,"diskio_usage":null}
{"date":"2022-08-09","cpu_usage":6.95,"ram_usage":62.01,"diskio_usage":null}
{"date":"2022-08-10","cpu_usage":6.91,"ram_usage":62.33,"diskio_usage":null}
{"date":"2022-08-11","cpu_usage":6.81,"ram_usage":61.52,"diskio_usage":null}
{"date":"2022-08-12","cpu_usage":7.5,"ram_usage":63.4,"diskio_usage":null}
{"date":"2022-08-15","cpu_usage":5.93,"ram_usage":64.43,"dis

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:

  Summarize the following table data.

  {"date":"2022-08-02","cpu_usage":7.85,"ram_usage":52.66,"diskio_usage":null}
{"date":"2022-08-03","cpu_usage":5.03,"ram_usage":53.52,"diskio_usage":null}
{"date":"2022-08-04","cpu_usage":4.86,"ram_usage":53.66,"diskio_usage":null}
{"date":"2022-08-05","cpu_usage":8.01,"ram_usage":55.25,"diskio_usage":null}
{"date":"2022-08-07","cpu_usage":16.91,"ram_usage":54.22,"diskio_usage":null}
{"date":"2022-08-08","cpu_usage":6.97,"ram_usage":59.46,"diskio_usage":null}
{"date":"2022-08-09","cpu_usage":6.95,"ram_usage":62.01,"diskio_usage":null}
{"date":"2022-08-10","cpu_usage":6.91,"ram_usage":62.33,"diskio_usage":null}
{"date":"2022-08-11","cpu_usage":6.81,"ram_usage":61.52,"diskio_usage":null}
{"date":"2022-08-12","cpu_usage":7.5,"ram_usage":63.4,"diskio_usage":null}
{"date":"2022-08-15","cpu_usage":5.93,"ram_usage":64.43,"dis

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:

  Summarize the following table data.

  {"date":"2022-08-02","cpu_usage":7.85,"ram_usage":52.66,"diskio_usage":null}
{"date":"2022-08-03","cpu_usage":5.03,"ram_usage":53.52,"diskio_usage":null}
{"date":"2022-08-04","cpu_usage":4.86,"ram_usage":53.66,"diskio_usage":null}
{"date":"2022-08-05","cpu_usage":8.01,"ram_usage":55.25,"diskio_usage":null}
{"date":"2022-08-07","cpu_usage":16.91,"ram_usage":54.22,"diskio_usage":null}
{"date":"2022-08-08","cpu_usage":6.97,"ram_usage":59.46,"diskio_usage":null}
{"date":"2022-08-09","cpu_usage":6.95,"ram_usage":62.01,"diskio_usage":null}
{"date":"2022-08-10","cpu_usage":6.91,"ram_usage":62.33,"diskio_usage":null}
{"date":"2022-08-11","cpu_usage":6.81,"ram_usage":61.52,"diskio_usage":null}
{"date":"2022-08-12","cpu_usage":7.5,"ram_usage":63.4,"diskio_usage":null}
{"date":"2022-08-15","cpu_usage":5.93,"ram_usage":64.43,"dis